**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install torchinfo
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from class_utils.pytorch_utils import BestModelCheckpointer, freeze_except_last
from torch.optim.lr_scheduler import ExponentialLR
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchinfo
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/w4pg809npvatye0/food5v2.zip?dl=1", directory="data/food5v2")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Transfer učenie

Transfer učenie budeme realizovať na dátovej množine **Food 5** : zmenšenej verzii dátovej množiny [Food 11](https://www.kaggle.com/vermaavi/food11). Iný príklad transfer učenia možné nájsť v [interaktívnom deme](https://storage.googleapis.com/tfjs-examples/webcam-transfer-learning/dist/index.html) pre tensorflow.js.

Transfer učenie je veľmi užitočná technika. Za normálnych okolností si hlboké učenie vyžaduje obrovské množstvo dát a výpočtov. Ak ho chceme aplikovať na malú dátovú množinu, typicky sa nám nepodarí dosiahnuť, aby hlboká sieť dobre zovšeobecňovala. Problém súvisí s tým, že malá dátová množina nedokáže väčšinou dostatočne vystihnúť všetky možné variácie vzoriek, s ktorými sa je možné stretnúť. Povedzme v prípade rozpoznávania obrazu môže existovať v podstate nekonečný počet variácií fotografie psa: líšiť sa môžu prostredím, osvetlením, plemenom psa, uhlom, v ktorom je odfotografovaný a pod. Malá dátová množina potom s vysokou pravdepodobnosťou nepokryje dostatočne tento široký priestor.

Jedným z riešení, ktoré umožňujú hlboké učenie predsa len aplikovať aj na pomerne malé dátové množiny, je **transfer učenie** . Ide o techniku, kde sa sieť najprv predtrénuje na veľkej, všeobecnejšej dátovej množine (v prípade spracovania obrazu to býva dátová množina ImageNet) – tam sa sieť naučí napríklad o tom, ako vyzerá prirodzený obraz a ako ho treba predspracovať. Následne sa už existujúca sieť dotrénuje na konkrétnu cieľovú úlohu.

### Rámcový postup

Rámcový postup transfer učenia pre spracovanie obrazu:

* Predtrénovať sieť na dátovej množine ImageNet.


* Z pôvodnej siete zmazať niekoľko posledných vrstiev a nahradiť ich novými. Nová výstupná vrstva bude už mať toľko výstupov, koľko je tried v novej dátovej množine.


* Váhy predtrénovaných vrstiev sa zafixujú. Na novej dátovej množine sa trénujú najprv len nové vrstvy.


* Keď sa nové vrstvy natrénovali, môžeme (nepovinný krok) odomknúť aj váhy predtrénovaných vrstiev a doladíme váhy celej siete. Použijeme omnoho nižšiu rýchlosť učenia – jednak preto, aby sme váhy priveľkými krokmi nerozladili, ale aj preto, že pri ladení všetkých váh sa už sieť veľmi ľahko preučí.


### Príprava dátovej množiny

Ako zvyčajne, začneme prípravou dátovej množiny. Pre väčšinu úloh rozpoznávania obrazu bude dátová množina príliš veľká na to, aby sa do pamäte zmestila celá naraz. Preto sa ju typicky nebudeme snažiť načítať celú naraz, ale využijeme `DataSet` a `DataLoader` abstrakcie z balíčka `PyTorch`. V našom aktuálnom príklade je dátová množina už predrozdelená na tréningové, validačné a testovacie dáta, pričom každá časť dát je uložená v osobitnom priečinku. Priečinky sú štruktúrované tak, že každá trieda má svoj vlastný podpriečinok. 



In [ ]:
!ls data/food5v2

In [ ]:
!ls data/food5v2/training

Keďže má dátová množina takúto štruktúru, môžeme použiť priamo triedu `ImageFolder` z `torchvision.datasets`.

Každý obrázok bude potrebné predtým, ako ho vložíme na vstup neurónovej siete, predspracovať: bude potrebné upraviť jeho rozmery, orezať ho a normalizovať rovnakým spôsobom ako sa to dialo keď bola sieť predtrénovaná. Budeme používať predtrénovanú sieť ResNet50 s váhami `IMAGENET1K_V2`. Pozrime sa teda najprv, ako vyzerá postup predspracovania, s ktorým boli trénované tieto váhy.



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
weights = models.ResNet50_Weights.IMAGENET1K_V2
image_transforms = weights.transforms()
image_transforms

Toto predspracovanie je vcelku jednoduché. Vytvoríme na základe neho dva rozličné spôsoby predspracovania pre naše dáta. Prvý z nich bude len reprodukovať `image_transforms`, ako sme ho videli vyššie. Druhý však bude vykonávať aj **zväčšovanie dátovej množiny**  – bude obsahovať niekoľko náhodných operácií, ktoré upravia obrázok iným spôsobom pri každom načítaní. Toto pridá do našej tréningovej množiny viac rozmanitosti. Sieť v podstate nikdy neuvidí ten istý obrázok dvakrát. V praxi môžu byť procedúry na zväčšovanie dátovej množiny oveľa prepracovanejšie, napr. na obraz aplikovať rotáciu, priblíženie, posun kanálov a množstvo ďalších transformácií.



In [ ]:
normal_preproc = transforms.Compose([
    transforms.Resize(image_transforms.resize_size),
    transforms.CenterCrop(image_transforms.crop_size),
    transforms.ToTensor(),
    transforms.Normalize(image_transforms.mean, image_transforms.std)
])

augment_preproc = transforms.Compose([
    transforms.RandomResizedCrop(image_transforms.crop_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(image_transforms.mean, image_transforms.std)
])

Ďalej už môžeme zostrojiť samotné `ImageFolder` dátové množiny. Špecifikujeme cesty ku jednotlivým častiam dátovej množiny a tiež spôsob, akým sa majú obrázky pre každú časť predspracovať. Bežné predspracovanie aplikujeme na validačné a testovacie dáta a predspracovanie so zväčšovaním dátovej množiny aplikujeme na tréningové dáta.



In [ ]:
train_dataset = ImageFolder(
    "data/food5v2/training",
    augment_preproc
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

valid_dataset = ImageFolder(
    "data/food5v2/validation",
    normal_preproc
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

test_dataset = ImageFolder(
    "data/food5v2/testing",
    normal_preproc
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

#### Zobrazenie niekoľkých vzoriek



In [ ]:
#@title -- Display Data Samples --
disp_dataset = ImageFolder(
    "data/food5v2/training",
    transforms.ToTensor()
)
loader = DataLoader(disp_dataset, batch_size=1, shuffle=True)
loader_iter = iter(loader)

num_rows = 4; num_cols = 4
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))

for row in axes:
    for ax in row:
        sample = next(loader_iter)[0][0].numpy().transpose((1, 2, 0))
        ax.imshow(sample)
        ax.set_xticks([])
        ax.set_yticks([])

### Načítanie predtrénovanej siete

Načítame predtrénovanú sieť s architektúrou ResNet50. Váhy predtrénované na dátovej množine ImageNet sa stiahnu automaticky.



In [ ]:
model = models.resnet50(weights=weights)

Aby sme získali predstavu o tom, ako vyzerá naša architektúra, použijeme funkciu `torchinfo.summary`. Poskytne nám informácie o hierarchickej štruktúre našej siete vrátane všetkých jej podmodulov a jednotlivých vrstiev. Zhrnutie úplne dole tiež ukazuje, koľko trénovateľných parametrov sieť obsahuje.



In [ ]:
torchinfo.summary(model)

### Modifikácia siete

#### Nahradenie poslednej vrstvy

Aby sme neurónovú sieť adaptovali na novú klasifikačnú úlohu, nahradíme jej poslednú vrstvu (úplne prepojenú lineárnu vrstvu `model.fc`) novým modulom.



In [ ]:
class ModelTop(nn.Module):
    def __init__(self, num_features, num_outputs):
        super().__init__()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_features, num_outputs)

    def set_dropout(self, p):
        self.dropout.p = p
    
    def forward(self, x):
        y = torch.flatten(x, 1)
        y = self.dropout(y)
        y = self.fc(y)
        return y

In [ ]:
num_features = model.fc.in_features
top = ModelTop(num_features=num_features, num_outputs=10)
model.fc = top
model.to(device);

### Tréning nových vrstiev

V tréningovej slučke použijeme checkpointovanie najlepších verzií modelu: budeme monitorovať validačnú chybu a zakaždým, keď sa zlepší, model uložíme. Na konci tréningu potom načítame späť najlepší uložený model.



#### Zmrazenie predtrénovaných vrstiev

Pripomeňme, že na začiatku chceme trénovať len nové vrchné vrstvy a predtrénované vrstvy chceme ponechať v pôvodnom stave. V našom prípade budeme preto musieť uzamknúť všetky vrstvy okrem poslednej. Použijeme na to preddefinovanú pomocnú funkciu, tá však interne len prejde po vrstvách a nastaví príslušným spôsobom `requires_grad` atribút pre všetky ich parametre.



In [ ]:
freeze_except_last(model);

Teraz si znova zobrazme sumár modelu, aby sme sa uistili, že všetko prebehlo správne. Mali by sme vidieť, že počet trénovateľných parametrov je teraz podstatne nižší (len parametre finálnej vrstvy v našom novom module) a pribudlo veľa netrénovateľných (zmrazených) parametrov. Všimnite si tiež, že pre zmrazené vrstvy je teraz počet parametrov zobrazený v zátvorkách – týmto spôsobom môžete skontrolovať, či ste zmrazili správne vrstvy.



In [ ]:
torchinfo.summary(model)

Aj keď na konci siete používame iba jednu lineárnu vrstvu pre 10 výstupných tried, stále máme dosť veľa trénovateľných parametrov: 20 490, ak používame ResNet50. Vzhľadom na to, že naša tréningová množina obsahuje len 200 vzoriek, je to obrovské množstvo parametrov. Dropout by mal so zovšeobecnením čiastočne pomôcť, no aj tak nemôžeme očakávať zázraky. Pre tento druh úlohy by nebolo ťažké získať viac dát – nechceme však, aby učenie v notebook-u trvalo príliš dlho, a preto pracujeme s malou dátovou množinou.



### Tréning nových vrstiev

V tréningovej slučke použijeme checkpointovanie najlepších verzií modelu: budeme monitorovať validačnú chybu a zakaždým, keď sa zlepší, model uložíme. Na konci tréningu potom načítame späť najlepší uložený model.



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
schedule = ExponentialLR(optimizer, gamma=0.9)
checkpointer = BestModelCheckpointer(checkpoint_path="output/best_model.pt")
loss_train = []
loss_valid = []

for epoch in range(30):
    epoch_train_loss = []
    epoch_valid_loss = []

    model.train()
    for X_batch, Y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        y_batch = model(X_batch)
        loss = criterion(y_batch, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_train_loss.append(loss.item())

    loss_train.append(np.mean(epoch_train_loss))

    model.eval()
    for X_batch, Y_batch in valid_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        with torch.no_grad():
            y_batch = model(X_batch)
            loss = criterion(y_batch, Y_batch)

        epoch_valid_loss.append(loss.item())

    loss_valid.append(np.mean(epoch_valid_loss))
    checkpointer(loss_valid[-1], model)
    schedule.step()

    if epoch % 5 == 0:
        print(f"epoch {epoch}, train loss: {np.mean(loss_train[-5:])}, valid loss: {np.mean(loss_valid[-5:])}")

print(f"epoch {epoch}, loss: {loss_train[-1]}")

In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

#### Hodnotenie modelu na validačných dátach

Ďalej si načítame z checkpoint-ového súboru najlepší uložený model a spustíme evaluáciu. Keďže sme s tréningom modelu ešte neskončili, budeme zatiaľ testovať len na **validačnej množine, nie na testovacej množine** .



In [ ]:
model.load_state_dict(torch.load("output/best_model.pt"));

In [ ]:
eval_Y = []
eval_y = []

model.eval()
for X_batch, Y_batch in valid_dataloader:
    eval_Y.extend(Y_batch.numpy())
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    
    with torch.no_grad():
        y_batch = model(X_batch)

    eval_y.extend(y_batch.argmax(dim=1).cpu().numpy())

eval_Y = np.array(eval_Y)
eval_y = np.array(eval_y)

cm = pd.crosstab(
    eval_Y, eval_y,
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, '\n')

acc = accuracy_score(eval_Y, eval_y)
print("Accuracy = {}".format(acc))

### Dolaďovanie (fine-tuning) predtrénovaných váh

Keď už natrénujeme nové záverečné vrstvy modelu, často má zmysel rozmraziť ešte niekoľko ďalších vrstiev siete a doladiť aj ich váhy. Zvyčajne sa však v tom prípade rýchlosť učenia výrazne zníži a namiesto agresívnejších optimalizátorov, ako je napríklad Adam, sa zväčša používa konzervatívnejší optimalizátor, napríklad `SGD`. Cieľom je zabezpečiť, aby kroky, ktoré vykoná optimalizátor, nenarušili predtrénované príznaky a nezmazali tým celý efekt transfer učenia. Za zmienku stojí, že ani v tomto štádiu sa zvyčajne nerozmrazí úplne celá sieť.

V našom prípade máme veľmi malé množstvo dát a je nepravdepodobné, že táto etapa dolaďovania skutočne pomôže zlepšiť výsledky. Môžeme sa však o to aspoň pokúsiť. Začneme tým, že rozmrazíme posledných 5 vrstiev.



In [ ]:
freeze_except_last(model, num_last=5);
torchinfo.summary(model)

---
### Úloha 1: Realizujte doladenie váh

**Následne modifikujte vyššie použitú tréningovú slučku na doladenie váh. Namiesto metódy `Adam` použite ako optimalizátor `SGD` a rýchlosť učenia nastavte na nižšiu hodnotu, napríklad 1e-7. Modifikujte aj checkpoint_path tak, aby checkpointy boli uložené v inom súbore, než predtým. Ak doladený model nepredstavuje zlepšenie oproti predchádzajúcej verzii, obnovte váhy predchádzajúcej verzie z príslušného checkpointu.** 

---


In [ ]:


# ----



#### Testovanie doladeného modelu

Teraz načítame späť najlepšiu verziu nášho doladeného modelu a vyhodnotíme ju. Keďže máme veľmi málo dát, je pravdepodobné, že výsledky nebudú lepšie než u verzie, kde sme trénovali len nové vrstvy.



In [ ]:
model.load_state_dict(torch.load("output/best_full_model.pt"));

In [ ]:
eval_Y = []
eval_y = []

model.eval()
for X_batch, Y_batch in valid_dataloader:
    eval_Y.extend(Y_batch.numpy())
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    
    with torch.no_grad():
        y_batch = model(X_batch)

    eval_y.extend(y_batch.argmax(dim=1).cpu().numpy())

eval_Y = np.array(eval_Y)
eval_y = np.array(eval_y)

cm = pd.crosstab(
    eval_Y, eval_y,
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, '\n')

acc = accuracy_score(eval_Y, eval_y)
print("Accuracy = {}".format(acc))

Teraz môžeme model ohodnotiť aj na testovacej množine. Na týchto konkrétnych dátach môžeme očakávať, že výsledky na testovacej časti budú v skutočnosti o niečo lepšie – testovacia množina sa v tomto prípade náhodou zdá byť o niečo menej náročná, čo sa môže stať, keď pracujete s veľmi malými množstvami dát.



In [ ]:
eval_Y = []
eval_y = []

model.eval()
for X_batch, Y_batch in test_dataloader:
    eval_Y.extend(Y_batch.numpy())
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    
    with torch.no_grad():
        y_batch = model(X_batch)

    eval_y.extend(y_batch.argmax(dim=1).cpu().numpy())

eval_Y = np.array(eval_Y)
eval_y = np.array(eval_y)

cm = pd.crosstab(
    eval_Y, eval_y,
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, '\n')

acc = accuracy_score(eval_Y, eval_y)
print("Accuracy = {}".format(acc))